<a href="https://colab.research.google.com/github/zxf000000/AmazonProj/blob/main/faceswap_trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Faceswap Trainer Notebook

**Description:**

This is a jupyter notebook for training faceswap models on Google Colaboratory or colab. You will get a NVIDIA K80 GPU with 12GB of VRAM available for traning. It's advised that you use an external storage to store your models like Google Drive so you don't loose your work if something happens to the colab instance. You should have the traning image files on the colab machine to avoid network syncing. The instructions are based on my personal workflow but you can change where or how you save and load your data.

**Instruction:**

You should check that your instance has a GPU available.

*   Go to top menu Runtime -> Change runtime type -> set Python3 and GPU
*   When done run the GPU status notebook to check GPU information

In the import packages section, uncomment the version of faceswap you would like to add. The first one is the latest faceswap code others are there for legacy reasons, version we know that work.

Connect your Google Drive with the colab, this will mount your Google Drive to folder Drive, it will act as a local folder

Alternativly you can upload images and models in zip files. Image set should be named set.zip and models models.zip. Run a cell and then browse the appropriate zip file.

Run the first cell in the faceswap traning section, that would initialize the image preview python functions

Run train compand with appopriate params:


*   -A - face A folder
*   -B - face B folder
*   -m - model path
*   -t - model type name, trainers: original, dfaker, dfl_h128, iae, unblanced, villain
*   -bs - batch size, use an approprite batch size based on what are you trying to achive and the available GPU memory
*   -s - save interavl, detemines how often does the model save and updates the preview window. Syncing with cloud storage can be slow with large models

If you are an advanced user, you can view the tran config ini file by runing the cell under traning. Cell under that one will update the config file with the cell contents

To end traning press the stop icon once, that will start the end process. Wait for the model to save after that the cell will stop executing

**Notes:**


*    Colab instance will shutdown after 12hours
*    If you get an error no GPU available try again later
*    You can shutdown the browser when executing a script after about an hour without the browser the notebook will terminate
*    If saving to Google Drive you have to monitor the amount of free space on the drive, every save cycle it will send prevous model to the trash. If the space fills up the last save is going to get corrupted. The training will break and a model from the trash will have be restored
*    You can run only 1 cell at a time


# Get GPU status


In [1]:
#@title Display Virtual Machine system information, use this to check the assigned GPU

def install_dependencies():
  !ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi;
  !pip install gputil;
  !pip install psutil;
  !pip install humanize;%%capture


def printm():
 GPUs = GPU.getGPUs()

 if len(GPUs) == 0:
  print("No GPU available.")
  return

 gpu = GPUs[0]
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))

from IPython.utils import io
from google.colab import drive
import psutil
import humanize
import os

with io.capture_output() as captured:
  install_dependencies()
print("Dependencies installed.")


import GPUtil as GPU

printm()

Dependencies installed.
Gen RAM Free: 87.8 GB  | Proc size: 106.9 MB
GPU RAM Free: 40506MB | Used: 0MB | Util   0% | Total 40960MB


# Import Packages

In [5]:
#@title Clone Faceswap github repository

!rm -rf faceswap
!git clone https://github.com/deepfakes/faceswap.git faceswap

# !pip install tensorflow-gpu==1.15.0
!pip install -r faceswap/requirements.txt

Cloning into 'faceswap'...
remote: Enumerating objects: 14682, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 14682 (delta 19), reused 16 (delta 16), pack-reused 14650 (from 2)
Receiving objects: 100% (14682/14682), 198.70 MiB | 43.63 MiB/s, done.
Resolving deltas: 100% (10480/10480), done.
ERROR: Could not open requirements file: [Errno 2] No such file or directory: './faceswap/requirements.txt'


# Mount Google Drive folders
---

Run and paste the code account code



In [ ]:
#@title Mount Google Drive only

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Load local files upload


In [ ]:
from google.colab import files
import os

local_download_path = os.path.expanduser('~/face1')
try:
  os.makedirs(local_download_path)
except: pass
!cd face1

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
#!bash -c 'mv *.zip /content/face1'
!unzip set.zip -d face1
!rm set.zip
!ls face1


In [ ]:
from google.colab import files
import os

!rm -rf face2
local_download_path = os.path.expanduser('~/face2')
try:
  os.makedirs(local_download_path)
except: pass

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
#!bash -c 'mv *.jpg /content/face2'

!unzip set.zip -d face2
!rm set.zip

!ls face2

In [ ]:
from google.colab import files
import os

!rm originalHighRes.zip
local_download_path = os.path.expanduser('~/models')
try:
  os.makedirs(local_download_path)
except: pass
#!cd models

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
!unzip models.zip -d models
!rm models.zip
!ls models

# Faceswap Training


In [ ]:
#Threading stuff stolen from:
# https://stackoverflow.com/questions/32081926/a-new-thread-for-running-a-cell-in-ipython-jupyter-notebook

from time import sleep,time
from IPython.display import display,HTML
import base64
from threading import Thread

# new faceswap version
image_path = "faceswap/training_preview.jpg"

# old faceswap
#image_path = "faceswap/_sample_training.jpg"
#image_path = "faceswap/_sample_training using <OriginalHighRes: v=2.7, enc=ORIGINAL, encoder_dim=1024, img_shape=128x128>, bs=16.jpg"
!touch '{image_path}'

class PreviewImg:
  def __init__(self):
    self.cancel = False
    self.fig = display(HTML('<img src="https://dummyimage.com/1024x618/000/ffffff&text=Preview+Loading..." />'), display_id=True)
    print("Created display.")

  def update(self):
    print("Updating display.")
    with open(image_path, 'rb') as image:
      encoded = str(base64.b64encode(image.read()))[2:-1]
      self.fig.update(HTML('<img src="data:image/jpg;base64,' + encoded + '" width="1024" height="618" />'))
      # print("Updated display.")

  def task(self):
    while not self.cancel:
      sleep(5 * 60) #Every 5 minutes.
      if not self.fig:
        print("Figure doesn't exist.")
        break
      self.update()

In [ ]:
!echo "Preview image size is reduced 3 times right click and save the image to view it in full size"

prev_img = PreviewImg()
thread = Thread(target=prev_img.task)
thread.start()

!python3 faceswap/faceswap.py train -A 'face1' -B 'face2' -m '/content/drive/My Drive/models' -t 'villain' -bs 16 -s 330 -w -nl

In [ ]:
# uncomment if you have local models or files/commented are examples
# first line display contents of the folder models orther lines are for downloads

#!ls models
#files.download('models/decoder_A.h5')
#files.download('models/decoder_B.h5')
#files.download('models/encoder.h5')
#files.download('models/decoder_A.h5.bk')
#files.download('models/decoder_B.h5.bk')
#files.download('models/encoder.h5.bk')

In [ ]:
# Shows traning config file
# Paste to cell under and run to reset the config
!cat faceswap/config/train.ini

In [ ]:
# Run to update the model config file

config = """
[global]
# OPTIONS THAT APPLY TO ALL MODELS

# Use ICNR Kernel Initializer for upscaling.
# This can help reduce the 'checkerboard effect' when upscaling the image.
# Choose from: True, False
# [Default: False]
icnr_init = False

# Use subpixel upscaling rather than pixel shuffler.
# Might increase speed at cost of VRAM
# Choose from: True, False
# [Default: False]
subpixel_upscaling = False

# Use reflect padding rather than zero padding.
# Choose from: True, False
# [Default: False]
reflect_padding = False

# If using a mask, Use DSSIM loss for Mask training rather than Mean Absolute Error
# May increase overall quality.
# Choose from: True, False
# [Default: True]
dssim_mask_loss = True

# If using a mask, Use Penalized loss for Mask training. Can stack with DSSIM.
# May increase overall quality.
# Choose from: True, False
# [Default: True]
penalized_mask_loss = True

[model.dfaker]
# DFAKER MODEL (ADAPTED FROM HTTPS://GITHUB.COM/DFAKER/DF)

# The mask to be used for training. Select none to not use a mask
# Choose from: ['none', 'dfaker', 'dfl_full']
# [Default: dfaker]
mask_type = dfaker

# How much of the extracted image to train on. Generally the model is optimized
# to the default value. Sensible values to use are:
# 	62.5%% spans from eyebrow to eyebrow.
# 	75.0%% spans from temple to temple.
# 	87.5%% spans from ear to ear.
# 	100.0%% is a mugshot.
# Select a decimal number between 62.5 and 100.0
# [Default: 100.0]
coverage = 100.0

[model.dfl_h128]
# DFL H128 MODEL (ADAPTED FROM HTTPS://GITHUB.COM/IPEROV/DEEPFACELAB)

# Lower memory mode. Set to 'True' if having issues with VRAM useage.
# NB: Models with a changed lowmem mode are not compatible with each other.
# Choose from: True, False
# [Default: False]
lowmem = False

# The mask to be used for training. Select none to not use a mask
# Choose from: ['none', 'dfaker', 'dfl_full']
# [Default: dfl_full]
mask_type = dfl_full

# How much of the extracted image to train on. Generally the model is optimized
# to the default value. Sensible values to use are:
# 	62.5%% spans from eyebrow to eyebrow.
# 	75.0%% spans from temple to temple.
# 	87.5%% spans from ear to ear.
# 	100.0%% is a mugshot.
# Select a decimal number between 62.5 and 100.0
# [Default: 62.5]
coverage = 62.5

[model.iae]
# INTERMEDIATE AUTO ENCODER. BASED ON ORIGINAL MODEL, USES INTERMEDIATE LAYERS TO TRY TO BETTER GET DETAILS

# Use DSSIM for Loss rather than Mean Absolute Error
# May increase overall quality.
# Choose from: True, False
# [Default: False]
dssim_loss = False

# The mask to be used for training. Select none to not use a mask
# Choose from: ['none', 'dfaker', 'dfl_full']
# [Default: none]
mask_type = none

# How much of the extracted image to train on. Generally the model is optimized
# to the default value. Sensible values to use are:
# 	62.5%% spans from eyebrow to eyebrow.
# 	75.0%% spans from temple to temple.
# 	87.5%% spans from ear to ear.
# 	100.0%% is a mugshot.
# Select a decimal number between 62.5 and 100.0
# [Default: 62.5]
coverage = 62.5

[model.original]
# ORIGINAL FACESWAP MODEL

# Lower memory mode. Set to 'True' if having issues with VRAM useage.
# NB: Models with a changed lowmem mode are not compatible with each other.
# Choose from: True, False
# [Default: False]
lowmem = False

# Use DSSIM for Loss rather than Mean Absolute Error
# May increase overall quality.
# Choose from: True, False
# [Default: False]
dssim_loss = False

# The mask to be used for training. Select none to not use a mask
# Choose from: ['none', 'dfaker', 'dfl_full']
# [Default: none]
mask_type = none

# How much of the extracted image to train on. Generally the model is optimized
# to the default value. Sensible values to use are:
# 	62.5%% spans from eyebrow to eyebrow.
# 	75.0%% spans from temple to temple.
# 	87.5%% spans from ear to ear.
# 	100.0%% is a mugshot.
# Select a decimal number between 62.5 and 100.0
# [Default: 62.5]
coverage = 62.5

[model.unbalanced]
# AN UNBALANCED MODEL WITH ADJUSTABLE INPUT SIZE OPTIONS.
# THIS IS AN UNBALANCED MODEL SO B>A SWAPS MAY NOT WORK WELL

# Lower memory mode. Set to 'True' if having issues with VRAM useage.
# NB: Models with a changed lowmem mode are not compatible with each other. NB: lowmem will override cutom nodes and complexity settings.
# Choose from: True, False
# [Default: False]
lowmem = False

# Use DSSIM for Loss rather than Mean Absolute Error
# May increase overall quality.
# Choose from: True, False
# [Default: False]
dssim_loss = False

# The mask to be used for training. Select none to not use a mask
# Choose from: ['none', 'dfaker', 'dfl_full']
# [Default: none]
mask_type = none

# Number of nodes for decoder. Don't change this unless you know what you are doing!
# Select an integer between 512 and 4096
# [Default: 1024]
nodes = 1024

# Encoder Convolution Layer Complexity. sensible ranges: 128 to 160
# Select an integer between 64 and 1024
# [Default: 128]
complexity_encoder = 128

# Decoder A Complexity.
# Select an integer between 64 and 1024
# [Default: 384]
complexity_decoder_a = 384

# Decoder B Complexity.
# Select an integer between 64 and 1024
# [Default: 512]
complexity_decoder_b = 512

# Resolution (in pixels) of the image to train on.
# BE AWARE Larger resolution will dramatically increaseVRAM requirements.
# Make sure your resolution is divisible by 64 (e.g. 64, 128, 256 etc.).
# NB: Your faceset must be at least 1.6x larger than your required input size.
#     (e.g. 160 is the maximum input size for a 256x256 faceset)
# Select an integer between 64 and 512
# [Default: 128]
input_size = 128

# How much of the extracted image to train on. Generally the model is optimized
# to the default value. Sensible values to use are:
# 	62.5%% spans from eyebrow to eyebrow.
# 	75.0%% spans from temple to temple.
# 	87.5%% spans from ear to ear.
# 	100.0%% is a mugshot.
# Select a decimal number between 62.5 and 100.0
# [Default: 62.5]
coverage = 62.5

[model.villain]
# A HIGHER RESOLUTION VERSION OF THE ORIGINAL MODEL BY VILLAINGUY.
# EXTREMELY VRAM HEAVY. FULL MODEL REQUIRES 9GB+ FOR BATCHSIZE 16

# Lower memory mode. Set to 'True' if having issues with VRAM useage.
# NB: Models with a changed lowmem mode are not compatible with each other.
# Choose from: True, False
# [Default: False]
lowmem = False

# Use DSSIM for Loss rather than Mean Absolute Error
# May increase overall quality.
# Choose from: True, False
# [Default: False]
dssim_loss = True

# The mask to be used for training. Select none to not use a mask
# Choose from: ['none', 'dfaker', 'dfl_full']
# [Default: none]
mask_type = none

# How much of the extracted image to train on. Generally the model is optimized
# to the default value. Sensible values to use are:
# 	62.5%% spans from eyebrow to eyebrow.
# 	75.0%% spans from temple to temple.
# 	87.5%% spans from ear to ear.
# 	100.0%% is a mugshot.
# Select a decimal number between 62.5 and 100.0
# [Default: 62.5]
coverage = 75
"""

with open("faceswap/config/train.ini", "w") as text_file:
    text_file.write(config)

# MachineTube Repository Model Export

Models trained as Original can be exported to machinetube format. Models can be added to https://www.machine.tube/models
Run the first cell to get the conversion code.
Before runing the conversion cell set the model path



In [ ]:
!git clone https://github.com/transcranial/keras-js.git

In [ ]:
# set the path to the model files
model_path = 'Drive/models'

import os
mode_paths = [f for f in os.listdir(model_path) if os.path.splitext(f)[1][1:] == 'h5']

print (mode_paths)

for file in mode_paths:
  full_path = os.path.join(model_path, file)
  !python3 keras-js/python/encoder.py $full_path -q